+ Simulationsprozesse durch ``process`` realisiert
+ Prozesse werden in einem ``environment`` angesiedelt, über die sie interagieren

**Szenario**: Ein Drucker, der sich zu festen Zeiten in festen in zwei möglichen Zuständen befindet: 
+ Drucker arbeitet für die Dauer ``druckdauer``
+ Drucker wartet auf Druckauftrag ``wartezeit``

In [19]:
import simpy as si

def drucker(env):
    while True:
        print('starte Druckvorgang %d' % env.now)
        druckdauer = 2
        yield env.timeout(druckdauer)
        
        print('Wartezeit auf Druckauftrag beginnt zur Zeit %d' % env.now)
        wartezeit = 10 
        yield env.timeout(wartezeit)

Prozess ``drucker`` benötigt eine Referenz auf die Umgebung ``env``, um neue Ereignisse zu starten. 
+ Prozess als Endlosschleife angelegt
+ Sobald ``yield`` erreicht wird, gibt Prozess ``drucker``die Kontrolle an das Environment (Simulation) ab.
+ Durch ``timeout``-Aufruf mit Übergabe des entsprechenden Ereignisses (hier: ``druckdauer``und ``wartezeit``) wird der Simulation signalisiert, auf dieses Ereignis zu warten.
+ Ausgabe des aktuellen Zustands zur aktuellen Simulationszeit

In [20]:
# erzeuge Simulationsumgebung
env = si.Environment()
env.process(drucker(env))

#starte Simulation
env.run(until=60)

starte Druckvorgang 0
Wartezeit auf Druckauftrag beginnt zur Zeit 2
starte Druckvorgang 12
Wartezeit auf Druckauftrag beginnt zur Zeit 14
starte Druckvorgang 24
Wartezeit auf Druckauftrag beginnt zur Zeit 26
starte Druckvorgang 36
Wartezeit auf Druckauftrag beginnt zur Zeit 38
starte Druckvorgang 48
Wartezeit auf Druckauftrag beginnt zur Zeit 50


**Szenario**: Betrachte Druckerpool mit 4 Druckern. Jede Minute kommt ein neuer Druckauftrag an, dessen Ausführung jeweils 3 Minuten dauert.

In [21]:
def druckerpool (env, auftrag, drucker, ankunft, druckdauer):
    #simuliere ankommende druckaufträge
    yield env.timeout(ankunft)
    print('%s kommt an zur Zeit %d' % (auftrag, env.now)) # "Auftrag XxI kommt an zur Zeit yy"
    tankunft = env.now

    #fordere drucker an
    with drucker.request() as req:
        yield req
            
        #auftrag wird gedruckt msobald ressource frei
        wartezeit = env.now - tankunft
        print ('%s Druckbeginn zur Zeit %d (mit Wartezeit %d und Dauer %d)' % (auftrag, env.now, wartezeit, druckdauer)) # "Auftrag xx Druckbeginn zur Zeit yy" 
        yield env.timeout(druckdauer)
        print('%s Druckende zur Zeit %d' % (auftrag, env.now)) # "Auftrag xx Druckende zur Zeit yy"

+ mit request() wird Durcker zur Bearbeitung angefordert bzw. u.U. auf einen freien Drucker gewartet
+ wird request zusammen mit with angewendet, dann wird die angeforderte Ressource nach Bearbeitung eines Auftrags automatisch wieder freigegeben
+ Ressource mit FCFS-Disziplin beschäftigt
+ Ressourcen brauchen einen Verweis auf Simulationsumgebung und eine Kapazitätsangabe (später)

In [22]:
# initialisiere Simulationsumgebung
env = si.Environment()
drucker = si.Resource(env, capacity=4)

# betrachte die ersten 10 druckaufträge
for i in range(1,11):
    env.process(druckerpool(env, f'Auftrag {i}', drucker, i, 3))

#starte simulation
env.run()

Auftrag 1 kommt an zur Zeit 1
Auftrag 1 Druckbeginn zur Zeit 1 (mit Wartezeit 0 und Dauer 3)
Auftrag 2 kommt an zur Zeit 2
Auftrag 2 Druckbeginn zur Zeit 2 (mit Wartezeit 0 und Dauer 3)
Auftrag 3 kommt an zur Zeit 3
Auftrag 3 Druckbeginn zur Zeit 3 (mit Wartezeit 0 und Dauer 3)
Auftrag 4 kommt an zur Zeit 4
Auftrag 1 Druckende zur Zeit 4
Auftrag 4 Druckbeginn zur Zeit 4 (mit Wartezeit 0 und Dauer 3)
Auftrag 5 kommt an zur Zeit 5
Auftrag 2 Druckende zur Zeit 5
Auftrag 5 Druckbeginn zur Zeit 5 (mit Wartezeit 0 und Dauer 3)
Auftrag 6 kommt an zur Zeit 6
Auftrag 3 Druckende zur Zeit 6
Auftrag 6 Druckbeginn zur Zeit 6 (mit Wartezeit 0 und Dauer 3)
Auftrag 7 kommt an zur Zeit 7
Auftrag 4 Druckende zur Zeit 7
Auftrag 7 Druckbeginn zur Zeit 7 (mit Wartezeit 0 und Dauer 3)
Auftrag 8 kommt an zur Zeit 8
Auftrag 5 Druckende zur Zeit 8
Auftrag 8 Druckbeginn zur Zeit 8 (mit Wartezeit 0 und Dauer 3)
Auftrag 9 kommt an zur Zeit 9
Auftrag 6 Druckende zur Zeit 9
Auftrag 9 Druckbeginn zur Zeit 9 (mit Wa

**Szenario**: Druckerpool mit zwei Druckern. Zwei aufeinanderfolgende Druckaufträge kommen im zufälligen (ganzzahligen) Abstand zwischen 1 und 4 Minuten an. Ihre Ausführung dauert jeweils eine (ganzzahlige) Dauer zwischen 2 und 6 Minuten.

In [26]:
import random

# initialisiere Simulationsumgebung
env = si.Environment()
drucker = si.Resource(env, capacity=2)

t0 = 0
# betrachte die ersten 10 druckaufträge
for i in range(1,11):
    t0 += random.randint(1,4)
    env.process(druckerpool(env, f'Auftrag {i}', drucker, t0, random.randint(2,6)))

env.run()

Auftrag 1 kommt an zur Zeit 2
Auftrag 1 Druckbeginn zur Zeit 2 (mit Wartezeit 0 und Dauer 6)
Auftrag 2 kommt an zur Zeit 6
Auftrag 2 Druckbeginn zur Zeit 6 (mit Wartezeit 0 und Dauer 3)
Auftrag 3 kommt an zur Zeit 7
Auftrag 4 kommt an zur Zeit 8
Auftrag 1 Druckende zur Zeit 8
Auftrag 3 Druckbeginn zur Zeit 8 (mit Wartezeit 1 und Dauer 6)
Auftrag 5 kommt an zur Zeit 9
Auftrag 2 Druckende zur Zeit 9
Auftrag 4 Druckbeginn zur Zeit 9 (mit Wartezeit 1 und Dauer 6)
Auftrag 6 kommt an zur Zeit 13
Auftrag 3 Druckende zur Zeit 14
Auftrag 5 Druckbeginn zur Zeit 14 (mit Wartezeit 5 und Dauer 5)
Auftrag 7 kommt an zur Zeit 15
Auftrag 4 Druckende zur Zeit 15
Auftrag 6 Druckbeginn zur Zeit 15 (mit Wartezeit 2 und Dauer 6)
Auftrag 8 kommt an zur Zeit 17
Auftrag 9 kommt an zur Zeit 19
Auftrag 5 Druckende zur Zeit 19
Auftrag 7 Druckbeginn zur Zeit 19 (mit Wartezeit 4 und Dauer 5)
Auftrag 6 Druckende zur Zeit 21
Auftrag 8 Druckbeginn zur Zeit 21 (mit Wartezeit 4 und Dauer 6)
Auftrag 10 kommt an zur Zeit

## M/M/2 - Supermarktkasse

In [27]:
# mittlerer Ankunftsabstand a, mittlere Bediendauer b [sekunden]

a = 25
b = 45

def kassensystem(env, kunde, kasse, ankunft, kassierdauer):
    # simuliere ankommende Kunden
    yield env.timeout(ankunft)
    print('%f %s Ankunft in Warteschlange' % (env.now, kunde))
    tankunft = env.now

    # fordere Kasiervorgang an
    with kasse.request() as req:
        yield req
        wartezeit = env.now - tankunft
        print('%f %s Kassiervorgang beginnt (Wartezeit %f)' % (env.now, kunde, wartezeit))
        
        yield env.timeout(kassierdauer)
        print('%f %s Kassiervorgang endet' % (env.now, kunde))


In [28]:
env = si.Environment()
kasse = si.Resource(env, capacity=2)

t0 = 0
# betrachte die ersten 10 Kunden
for i in range(1,11):
    t0 += random.expovariate(1/a)
    env.process(kassensystem(env, f'Kunde {i}', kasse, t0, random.expovariate(1/b)))

env.run()

7.557243 Kunde 1 Ankunft in Warteschlange
7.557243 Kunde 1 Kassiervorgang beginnt (Wartezeit 0.000000)
9.293814 Kunde 1 Kassiervorgang endet
10.631187 Kunde 2 Ankunft in Warteschlange
10.631187 Kunde 2 Kassiervorgang beginnt (Wartezeit 0.000000)
21.058087 Kunde 3 Ankunft in Warteschlange
21.058087 Kunde 3 Kassiervorgang beginnt (Wartezeit 0.000000)
21.150267 Kunde 2 Kassiervorgang endet
25.221955 Kunde 3 Kassiervorgang endet
46.359273 Kunde 4 Ankunft in Warteschlange
46.359273 Kunde 4 Kassiervorgang beginnt (Wartezeit 0.000000)
48.645665 Kunde 4 Kassiervorgang endet
49.278245 Kunde 5 Ankunft in Warteschlange
49.278245 Kunde 5 Kassiervorgang beginnt (Wartezeit 0.000000)
93.395732 Kunde 6 Ankunft in Warteschlange
93.395732 Kunde 6 Kassiervorgang beginnt (Wartezeit 0.000000)
96.723005 Kunde 6 Kassiervorgang endet
158.610302 Kunde 5 Kassiervorgang endet
181.966087 Kunde 7 Ankunft in Warteschlange
181.966087 Kunde 7 Kassiervorgang beginnt (Wartezeit 0.000000)
187.425462 Kunde 8 Ankunft in W